In [12]:
import plotly
from algo import *

In [17]:
def change_HMS(t=0):
    '''change int to h-m-s format 
    '''
    if t < 0 or t > 999999:
        return None, None, None
    
    h, m, s= "{:02d}".format(int(t/10000)), "{:02d}".format(int(t%10000/100)), "{:02d}".format(int(t%100))

    return h, m, s

In [36]:
def change_to_HMS(t=''):
    '''change time format to int
    '''
    return int(str(int(t.hour))+"{:02d}".format(int(t.minute))+"{:02d}".format((int(t.second))))


In [38]:
import datetime
a = datetime.time(hour=1,minute=0,second=0)
print(change_to_HMS(a))

10000


In [18]:
# 获取几只代码的数据
main_future_codes = []
date = '2023-09-12'


In [19]:
def get_main_future_codes(end_date_string, period = 20, period_minute = 20, threshold = 0.05):

	end_date = datetime.strptime(end_date_string, "%Y-%m-%d")
	end_date_ahead = end_date - timedelta(days=1)
	end_date_ahead_string = end_date_ahead.strftime('%Y-%m-%d')  # 格式为 YYYY-MM-DD

	if is_valid_date(end_date_string):

		"""
			C1: 正基差过滤
		"""
		t1 = time.time()
		# get namelist by basis of percentage
		print(end_date_ahead_string)
		basis_future = pd.DataFrame([i for i in DCE_commodity_price_collection.find(
						# {"date": end_date_string}
						{"date": end_date_ahead_string}
					).sort("main_future_code", pymongo.ASCENDING) ])
		# print(basis_future)
		# print(basis_future)
		basis_future = basis_future.sort_values(by='main_cf_basis_percent', ascending=False)

		t2 = time.time()

		# Find the main contract with +5% positive basis 
		codes = basis_future[basis_future["main_cf_basis_percent"]>=threshold]["main_future_code"].tolist()
		codes = list(set(codes))
		return codes

In [20]:
main_future_codes = get_main_future_codes(date)
print(main_future_codes)

2023-09-11
['FU2401', 'RS2311', 'CF2401', 'LC2401', 'SA2401', 'RM2401', 'M2401', 'UR2401', 'I2401', 'CS2311', 'SR2401', 'C2311', 'Y2401', 'CY2401']


In [ ]:
# def get_main_future_price_daily(codes,date):
    

In [ ]:
def get_realtime_price(date, tt, codelist=[]):
    realtime_prices_df = pd.DataFrame()
    # print(codelist)
    for code in codelist:
        cur_realtime_prices = DCE_future_price_collection.find(
            {"$and":[
                {'date':date},
                {'clock':{"$lt":tt}},
                {'future_code':code}]}
            ).sort("clock", pymongo.DESCENDING).limit(1)
        cur_df = pd.DataFrame(list(cur_realtime_prices))
		# print(cur_df)
        realtime_prices_df = pd.concat([realtime_prices_df, cur_df], ignore_index=True)

    return realtime_prices_df

In [ ]:

"""
	Here, 'end_date_string' typically represents today. 
	However, 'end_date' should be adjusted to be one day ahead ('end_date_ahead') for easier calculations, 
	especially when computing indicators such as MA and KDJ.
"""
def get_name_list(end_date_string, tt, period = 20, period_minute = 20, threshold = 0.05):
	end_date = datetime.strptime(end_date_string, "%Y-%m-%d")
	end_date_ahead = end_date - timedelta(days=1)
	end_date_ahead_string = end_date_ahead.strftime('%Y-%m-%d')  # 格式为 YYYY-MM-DD

	if is_valid_date(end_date_string):

		"""
			C1: 正基差过滤
		"""
		t1 = time.time()
		# get namelist by basis of percentage
		print(end_date_ahead_string)
		basis_future = pd.DataFrame([i for i in DCE_commodity_price_collection.find(
						# {"date": end_date_string}
						{"date": end_date_ahead_string}
					).sort("main_future_code", pymongo.ASCENDING) ])
		# print(basis_future)
		print(basis_future)
		basis_future = basis_future.sort_values(by='main_cf_basis_percent', ascending=False)

		t2 = time.time()

		# Find the main contract with +5% positive basis 
		codes = basis_future[basis_future["main_cf_basis_percent"]>=threshold]["main_future_code"].tolist()
		codes = list(set(codes))

		"""
			C2: 价格高于20日均线，且日KDJ交金叉
		"""
		# A larger date range (2*period) to ensure there's no data shortage 
		# due to non-trading days on weekends (Saturday and Sunday).
		start_date_string = get_previous_date(date_string = end_date_string, period=period*2+1) 
		dates_list = generate_dates(start_date_string, end_date_string)

		# get the price information of main contract, including high,low, open,close
		daily_prices = get_daily_info(codes, dates_list, period) 
		# print(daily_prices.columns) #['_id', 'date', 'future_code', 'close', 'open', 'high', 'low', 'deal', 'categoryID'],
		
		t3 = time.time()
		# print(f'xxxxx')
		# print(end_date_string)
		realtime_prices = get_realtime_price(end_date_string, tt, codes)
		# realtime_prices.to_csv("./realtime_prices.csv", index=False, encoding="gbk")

		ma_selected_codes = check_upper_ma(daily_prices, realtime_prices, period)
		# print(ma_selected_codes)

		t4 = time.time()

		kdj_selected_codes = check_daily_kdj(daily_prices, ma_selected_codes)
		# kdj_selected_codes = check_daily_kdj(daily_prices, codes)
		# print(kdj_selected_codes)

		"""
			C3: 价格高于60分钟-20均线，且60分钟-KDJ交金叉
		"""
		t5 = time.time()

		# for fast verification
		if os.path.isfile("./minutes_df.csv"):
			minutes_df = pd.read_csv("./minutes_df.csv", encoding="gbk")
		else:
			minutes_df = get_minutes_info(codes, end_date_string)
			# minutes_df.to_csv("./minutes_df.csv", index=False, encoding="gbk")	
			print(len(minutes_df))

		t6 = time.time()

		hourly_prices = minutes_df.groupby('future_code').apply(\
			lambda group: group.iloc[::-60]).reset_index(drop=True)

		ma_selected_codes_hourly = check_upper_ma(hourly_prices, realtime_prices, period_minute, if_minute=True)
		# print(ma_selected_codes_hourly)

		kdj_selected_codes_hourly = check_hourly_kdj(minutes_df, ma_selected_codes_hourly)
		# print(kdj_selected_codes_hourly)
		t7 = time.time()

		# 创建空的 DataFrame
		final_codes = list(set(kdj_selected_codes) & set(kdj_selected_codes_hourly))
		final_results = pd.DataFrame(columns=['评估指标','评估结果'])
		final_results.loc[0] = ["正基差", codes]
		final_results.loc[1] = ["价格高于20日均线，且日KDJ交金叉", kdj_selected_codes]
		final_results.loc[2] = ["价格高于60分钟-20均线，且60分钟-KDJ交金叉", kdj_selected_codes]
		final_results.loc[3] = ["综合结果", final_codes]
		final_results.to_csv("./final_results.csv", index=False, encoding="gbk")

		# print(f"代码运行时间: {t2 - t1} 秒")
		# print(f"代码运行时间: {t3 - t2} 秒")
		# print(f"代码运行时间: {t4 - t3} 秒")
		# print(f"代码运行时间: {t5 - t4} 秒")
		# print(f"代码运行时间: {t6 - t5} 秒")
		# print(f"代码运行时间: {t7 - t6} 秒")
		# print("正基差： ",codes)
		# print("ma_selected_codes ",ma_selected_codes)
		# print("kdj_selected_codes ",kdj_selected_codes)
		# print("ma_selected_codes_hourly ",ma_selected_codes_hourly)
		# print("kdj_selected_codes_hourly ",kdj_selected_codes_hourly)
		# print("final_codes ", final_codes)
		# print(final_results)
		print(final_results)
		return final_results
	else:
		print("错误日期信息")
		return None